In [1]:
SCENARIOS = [
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.4.0\baseline",
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.5.0a2\baseline",
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.5.0a3\baseline",
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.5.0a3-new-environment\baseline"
]
SCENARIO_NAMES = ["3.4.0", "3.5.0a2", "3.5.0a3", "3.5.0a4"]

In [2]:
import cea.inputlocator
locators = [cea.inputlocator.InputLocator(s) for s in SCENARIOS]
locator = locators[0]

In [3]:
import cea.schemas
schemas = cea.schemas.schemas(plugins=[])

In [4]:
import yaml
schemas_yml = r"C:\Users\darthoma\Documents\GitHub\CityEnergyAnalyst\cea\schemas.yml"
with open(schemas_yml, "r") as fp:
    schemas = yaml.safe_load(fp)

In [5]:
SKIP = {
    "get_optimization_building_scale_heating_capacity", # ??
    "get_optimization_district_scale_heating_capacity", # ??
    "get_optimization_generation_building_scale_performance", # debug only
    "get_optimization_generation_district_scale_performance", # debug only
    "get_optimization_generation_total_performance", # debug only
    "get_optimization_network_results_summary", # barcode...
    "get_optimization_substations_results_file", # barcode
    "get_optimization_substations_total_file"
}

In [8]:
import warnings
warnings.simplefilter("ignore")

# arguments to use with the locator methods
default_args = {
    "building": "B1000",
    "network_type": "DH",
    "network_name": "",
    "panel_type": "ET",
    "use": "RESTAURANT",
    "format": "csv",
    "generation": 2,
    "representative_week": False,
    "ind_num": 0,
    "gen_num": 2,
    "configuration": "AHU_ARU_SCU",
}

In [6]:
def float_and_int_columns(schemas, lm):
    result = []
    for col in schemas[lm]["schema"]["columns"]:
        col_type = schemas[lm]["schema"]["columns"][col]["type"]
        if col_type in {"float", "int"}:
            result.append(col)
    return result

In [9]:
# Figure out the files that differ 
for lm in schemas.keys():
    if lm in SKIP:
        continue
        
    if not schemas[lm]["file_type"] in ("csv", "shp", "dbf"):
        # ignore excel files etc. for now...
        continue        
    
    print(f"Checking lm={lm}")
        
    args = {p: default_args[p] for p in schemas[lm]["parameters"]}
    sios = [getattr(loc, lm) for loc in locators]
    dfs = [sio.read(**args) for sio in sios]

    # folder to store the figures in
    figfolder = f"{lm}"
    figabsfolder = os.path.join(outpath, figfolder)    
    if not os.path.exists(figabsfolder):
        os.mkdir(figabsfolder)
        
    for column in float_and_int_columns(schemas, lm):
        title=f"{lm}.{column}"
        figname = f"{column}.png"            
        df = pd.DataFrame([df[column] for df in dfs], index=SCENARIO_NAMES)    
        print(f"...Saving {figname}")
        df.plot(title=title, legend=None)
        plt.savefig(os.path.join(figabsfolder, figname))
        plt.close()    

Checking lm=PVT_metadata_results
...Saving AREA_m2.png
...Saving B_deg.png
...Saving CATB.png
...Saving CATGB.png
...Saving CATteta_z.png
...Saving Xcoor.png
...Saving Xdir.png
...Saving Ycoor.png
...Saving Ydir.png
...Saving Zcoor.png
...Saving Zdir.png
...Saving area_installed_module_m2.png
...Saving array_spacing_m.png
...Saving intersection.png
...Saving surface_azimuth_deg.png
...Saving tilt_deg.png
...Saving total_rad_Whm2.png
Checking lm=PVT_results
...Saving Area_PVT_m2.png
...Saving E_PVT_gen_kWh.png
...Saving Eaux_PVT_kWh.png
...Saving PVT_roofs_top_E_kWh.png
...Saving PVT_roofs_top_Q_kWh.png
...Saving PVT_roofs_top_m2.png
...Saving PVT_walls_east_E_kWh.png
...Saving PVT_walls_east_Q_kWh.png
...Saving PVT_walls_east_m2.png
...Saving PVT_walls_north_E_kWh.png
...Saving PVT_walls_north_Q_kWh.png
...Saving PVT_walls_north_m2.png
...Saving PVT_walls_south_E_kWh.png
...Saving PVT_walls_south_Q_kWh.png
...Saving PVT_walls_south_m2.png
...Saving PVT_walls_west_E_kWh.png
...Saving PV

KeyError: 'SC_FP_roofs_top_Q_kWh'